In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning

## Homework 8: Introduction to Computer vision, Time Series, and Survival Analysis (Lectures 19 to 20) 

**Due date: see the [Apr 07, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points:2}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset and storing it under the `data` folder. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture demo, we had different measurements for each Location. 

We want you to consider this for the avocado prices dataset. For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [9]:
df.dtypes

Date            datetime64[ns]
AveragePrice           float64
Total Volume           float64
4046                   float64
4225                   float64
4770                   float64
Total Bags             float64
Small Bags             float64
Large Bags             float64
XLarge Bags            float64
type                    object
year                     int64
region                  object
dtype: object

In [10]:
num_regions = df["region"].nunique()
num_types = df["type"].nunique()

print("Number of unique regions:", num_regions)
print("Number of unique avocado types:", num_types)

Number of unique regions: 54
Number of unique avocado types: 2


In [11]:
grouped = df.groupby(["region", "type"]).size().reset_index(name="Count")
num_time_series = len(grouped)
num_time_series 

108

The avocado prices dataset includes separate measurements for each unique region and type since the region column identifies distinct geographic areas and the type column distinguishes between conventional and organic avocados. The dataset has 54 unique regions and 2 unique types so that every combination of these features represents its own time series which result in 108 separate time series in total.

In [12]:
df.sort_values(by=["region", "type", "Date"]).head(10) 

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
50,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
49,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
48,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
47,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
46,2015-02-08,0.99,51253.97,1357.37,39111.81,163.25,10621.54,10113.10,508.44,0.0,conventional,2015,Albany
45,2015-02-15,1.06,41567.62,986.66,30045.51,222.42,10313.03,9979.87,333.16,0.0,conventional,2015,Albany
44,2015-02-22,1.07,45675.05,1088.38,35056.13,151.00,9379.54,9000.16,379.38,0.0,conventional,2015,Albany
43,2015-03-01,0.99,55595.74,629.46,45633.34,181.49,9151.45,8986.06,165.39,0.0,conventional,2015,Albany
42,2015-03-08,1.07,40507.36,795.68,30370.64,159.05,9181.99,8827.55,354.44,0.0,conventional,2015,Albany


The dataset shows that each date is represented only once for every combination of region and type. This confirms that the measurements for each unique region and type pair are tracked independently over time.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [13]:
# Group by region and type and print out
for (region, avocado_type), group in df.groupby(["region", "type"]):
    sorted_dates = group["Date"].sort_values()
    date_diff = sorted_dates.diff().dropna()
    print("Group: {} - {} | Min gap: {} | Max gap: {}".format(region, avocado_type, date_diff.min(), date_diff.max()))

Group: Albany - conventional | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Albany - organic | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Atlanta - conventional | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Atlanta - organic | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: BaltimoreWashington - conventional | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: BaltimoreWashington - organic | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Boise - conventional | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Boise - organic | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Boston - conventional | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: Boston - organic | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: BuffaloRochester - conventional | Min gap: 7 days 00:00:00 | Max gap: 7 days 00:00:00
Group: BuffaloRochester - organic | Min gap: 7 days 00:00:00 | Max gap: 7 da

The avocado dataset generally has equally spaced measurements on a weekly basis. Most region and type groups show both a minimum and maximum gap of exactly seven days between consecutive dates, which indicates consistent weekly intervals. However, there is an exception: the WestTexNewMexico region with organic avocados where the maximum gap is 21 days. This suggests that although the dataset is almost always recorded weekly, some measurements may be missing or delayed in a few cases.

In [14]:
# Filter for WestTexNewMexico organic avocados
west_tex_org = df[(df["region"] == "WestTexNewMexico") & (df["type"] == "organic")].sort_values("Date")
diffs = west_tex_org["Date"].diff().dropna().value_counts()
diffs

Date
7 days     163
14 days      1
21 days      1
Name: count, dtype: int64

In [15]:
# For a single group, check how frequently each time gap occurs
sample_group = df[
    (df["region"] == "Albany") &
    (df["type"] == "conventional")
].sort_values("Date")
date_diffs = sample_group["Date"].diff().dropna()
print("Date difference frequencies for Albany, conventional:")
print(date_diffs.value_counts())


Date difference frequencies for Albany, conventional:
Date
7 days    168
Name: count, dtype: int64


In the example above, Albany conventional avocados consistently exhibit a 7-day gap. In contrast, WestTexNewMexico organic avocados occasionally jump by 14 or 21 days. This confirms that the measurements are almost always equally spaced but with a few exceptions.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [16]:
# Display the unique region names in the dataset
print("Unique regions:", df["region"].unique())

Unique regions: ['Albany' 'Atlanta' 'BaltimoreWashington' 'Boise' 'Boston'
 'BuffaloRochester' 'California' 'Charlotte' 'Chicago' 'CincinnatiDayton'
 'Columbus' 'DallasFtWorth' 'Denver' 'Detroit' 'GrandRapids' 'GreatLakes'
 'HarrisburgScranton' 'HartfordSpringfield' 'Houston' 'Indianapolis'
 'Jacksonville' 'LasVegas' 'LosAngeles' 'Louisville' 'MiamiFtLauderdale'
 'Midsouth' 'Nashville' 'NewOrleansMobile' 'NewYork' 'Northeast'
 'NorthernNewEngland' 'Orlando' 'Philadelphia' 'PhoenixTucson'
 'Pittsburgh' 'Plains' 'Portland' 'RaleighGreensboro' 'RichmondNorfolk'
 'Roanoke' 'Sacramento' 'SanDiego' 'SanFrancisco' 'Seattle'
 'SouthCarolina' 'SouthCentral' 'Southeast' 'Spokane' 'StLouis' 'Syracuse'
 'Tampa' 'TotalUS' 'West' 'WestTexNewMexico']


In [17]:
date = pd.Timestamp("2015-02-01")

In [18]:
df.query("region == 'TotalUS' and type == 'conventional' and Date == @date")["Total Volume"].values[0]

44655461.51

In [19]:
df.query("region != 'TotalUS' and type == 'conventional' and Date == @date")["Total Volume"].sum()

72836765.58

The region column includes names that refer to individual cities such as Albany and Boston and also to larger areas such as California and West. There appears to be a hierarchical structure in the data. TotalUS is an aggregate of all regions and it is split into bigger areas such as West, Southeast, Northeast, and Midsouth. Moreover, California is also further divided into cities such as Sacramento, SanDiego, and LosAngeles. This indicates that while each region label is distinct in the dataset, many of these labels overlap in their geographic coverage. For example, on 2015-02-01 the volume recorded for TotalUS was 44565461.51 while the sum of the volumes for other regions was 72836756.58. The fact that these two numbers are close in scale but not identical suggests that TotalUS aggregates data from the individual regions which shows that the regions are not entirely separate.

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2024W2/tree/main/lectures), with some improvements.

In [20]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [21]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [22]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [23]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 `AveragePrice` baseline 
rubric={points}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [24]:
train_r2 = r2_score(df_train["AveragePriceNextWeek"], df_train["AveragePrice"])
train_r2 

0.8285800937261841

In [25]:
test_r2  = r2_score(df_test["AveragePriceNextWeek"], df_test["AveragePrice"])
test_r2 

0.7631780188583048

The baseline model predicts that next week's average price is the same as this week's price. This simple strategy yields a train $R^2$ of about 0.83 and a test $R^2$ of around 0.76.


In [26]:
assert not train_r2 is None, "Are you using the correct variable name?"
assert not test_r2 is None, "Are you using the correct variable name?"
assert sha1(str(round(train_r2, 3)).encode('utf8')).hexdigest() == 'b1136fe2a8918904393ab6f40bfb3f38eac5fc39', "Your training score is not correct. Are you using the right features?"
assert sha1(str(round(test_r2, 3)).encode('utf8')).hexdigest() == 'cc24d9a9b567b491a56b42f7adc582f2eefa5907', "Your test score is not correct. Are you using the right features?"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [27]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

def preprocess_data(df_train, df_test, num_feats, cat_feats, keep_feats, drop_feats, target_col):
    # Reorder columns
    order = num_feats + cat_feats + keep_feats + drop_feats + [target_col]
    df_train = df_train[order]
    df_test  = df_test[order]
    
    # Build pipelines for numeric and categorical features
    num_pipe = Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('imp', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])
    
    transformer = ColumnTransformer([
        ('num', num_pipe, num_feats),
        ('cat', cat_pipe, cat_feats),
        ('keep', 'passthrough', keep_feats),
        ('drop', 'drop', drop_feats + [target_col])
    ])
    
    transformer.fit(df_train)
    
    # Get new feature names for categorical data
    ohe = transformer.named_transformers_['cat'].named_steps['ohe']
    cat_feature_names = list(ohe.get_feature_names_out(cat_feats))
    new_cols = num_feats + cat_feature_names + keep_feats
    
    # Transform data 
    X_train = pd.DataFrame(transformer.transform(df_train).toarray(), index=df_train.index, columns=new_cols)
    X_test  = pd.DataFrame(transformer.transform(df_test).toarray(), index=df_test.index, columns=new_cols)
    y_train = df_train[target_col]
    y_test  = df_test[target_col]
    
    return X_train, y_train, X_test, y_test


In [28]:
# Convert Date to datetime 
df_train.loc[:, "Date"] = pd.to_datetime(df_train["Date"])
df_test.loc[:, "Date"]  = pd.to_datetime(df_test["Date"])
df_train.loc[:, "year"] = df_train["Date"].dt.year
df_test.loc[:, "year"]  = df_test["Date"].dt.year

# Add Month as a categorical variable 
df_train = df_train.assign(Month = df_train["Date"].dt.month.astype(str))
df_test  = df_test.assign(Month  = df_test["Date"].dt.month.astype(str))

# Type and region are strings
df_train.loc[:, "type"] = df_train["type"].astype(str)
df_train.loc[:, "region"] = df_train["region"].astype(str)
df_test.loc[:, "type"] = df_test["type"].astype(str)
df_test.loc[:, "region"] = df_test["region"].astype(str)

In [29]:
# Define feature sets for Experiment 1
num_feats1    = ["Total Volume", "4046", "4225", "4770", "Small Bags", "Large Bags", "XLarge Bags", "year"]
cat_feats1    = ["type", "region", "Month"]  # Month as categorical
keep_feats1   = ["AveragePrice"]
drop_feats1   = ["Date", "Total Bags"]
target_col    = "AveragePriceNextWeek"

In [30]:
#Preprocess the data
X_train1, y_train1, X_test1, y_test1 = preprocess_data(df_train, df_test, 
                                                        num_feats1, 
                                                        cat_feats1, 
                                                        keep_feats1, 
                                                        drop_feats1, 
                                                        target_col)

In [31]:
# TimeSeriesSplit and grid search for Ridge Regression
tscv = TimeSeriesSplit(n_splits=5)
ridge_grid = {'alpha': [11, 12, 13]}
ridge_model = Ridge()
ridge_gs = GridSearchCV(ridge_model, ridge_grid, cv=tscv, scoring='r2', n_jobs=-1)
ridge_gs.fit(X_train1, y_train1)

print("-----Experiment 1: Month Encoding-----")
print("Ridge Best Params:", ridge_gs.best_params_)
print("Ridge Best CV R^2:", ridge_gs.best_score_)
print("Ridge Train R^2:", ridge_gs.score(X_train1, y_train1))
print("Ridge Test R^2:", ridge_gs.score(X_test1, y_test1))

-----Experiment 1: Month Encoding-----
Ridge Best Params: {'alpha': 13}
Ridge Best CV R^2: 0.8209426304102811
Ridge Train R^2: 0.8492502671064784
Ridge Test R^2: 0.8005850197548069


In [32]:
# TimeSeriesSplit and grid search for Random Forest
rf_grid = {
    'n_estimators': [155, 160, 165],
    'max_depth': [5, 6, 7],
    'min_samples_leaf': [3, 4, 5]
}

rf_model = RandomForestRegressor(random_state=30)
rf_gs = GridSearchCV(rf_model, rf_grid, cv=tscv, scoring='r2', n_jobs=-1)
rf_gs.fit(X_train1, y_train1)

print("RandomForest Best Params:", rf_gs.best_params_)
print("RandomForest Best CV R^2:", rf_gs.best_score_)
print("RandomForest Train R^2:", rf_gs.score(X_train1, y_train1))
print("RandomForest Test R^2:", rf_gs.score(X_test1, y_test1))

RandomForest Best Params: {'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 160}
RandomForest Best CV R^2: 0.8300186641979901
RandomForest Train R^2: 0.857863489092283
RandomForest Test R^2: 0.7832833193077353


In [33]:
# Print Ridge coefficients 
ridge_best1 = ridge_gs.best_estimator_
coef_df1 = pd.DataFrame({
    'Feature': X_train1.columns,
    'Coefficient': ridge_best1.coef_
})

coef_df1['AbsCoef'] = coef_df1['Coefficient'].abs()
print("Ridge Coefficients (Experiment 1):")
print(coef_df1.sort_values("AbsCoef", ascending=False))


Ridge Coefficients (Experiment 1):
                Feature  Coefficient   AbsCoef
73         AveragePrice     0.769744  0.769744
8          type_organic     0.115029  0.115029
26       region_Houston    -0.094677  0.094677
72              Month_9     0.093831  0.093831
50  region_SanFrancisco     0.086949  0.086949
..                  ...          ...       ...
3                  4770    -0.004102  0.004102
43        region_Plains    -0.003089  0.003089
55       region_Spokane    -0.002549  0.002549
51       region_Seattle    -0.000752  0.000752
0          Total Volume     0.000349  0.000349

[74 rows x 3 columns]


In Experiment 1 with month encoding, the Ridge regression model achieved a CV $R^2$ of 0.82, with training and test \( R^2 \) scores respectively of **0.85** and **0.80** and had best parameter: alpha = 13. The Random Forest model achieved a CV $R^2$ of **0.83**, with training and test scores of respectively **0.86** and **0.78** and had best parameters: max_depth=6, min_samples_leaf=4, and n_estimators=160. The Ridge coefficients indicated that the most influential feature was AveragePrice (0.77), followed by type_organic (0.12) and region_Houston (-0.09).

In [34]:
# Define feature sets for Experiment 2
num_feats2    = ["Total Volume", "4046", "4225", "4770", "Small Bags", "Large Bags", "XLarge Bags"]  # Remove year
cat_feats2    = ["type", "region", "year"]  # 'year' as categorical
keep_feats2   = ["AveragePrice"]
drop_feats2   = ["Date", "Total Bags", "Month"]  # Drop Month in this experiment


In [35]:
# Preprocess the data
X_train2, y_train2, X_test2, y_test2 = preprocess_data(df_train, df_test, 
                                                        num_feats2, 
                                                        cat_feats2, 
                                                        keep_feats2, 
                                                        drop_feats2, 
                                                        target_col)

C:\Users\junse\miniconda3\envs\cpsc330\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [36]:
# TimeSeriesSplit and grid search for Ridge Regression
ridge_model2 = Ridge()
ridge_gs2 = GridSearchCV(ridge_model2, ridge_grid, cv=tscv, scoring='r2', n_jobs=-1)
ridge_gs2.fit(X_train2, y_train2)

print("\n-----Experiment 2: Year Encoding-----")
print("Ridge Best Params:", ridge_gs2.best_params_)
print("Ridge Best CV R^2:", ridge_gs2.best_score_)
print("Ridge Train R^2:", ridge_gs2.score(X_train2, y_train2))
print("Ridge Test R^2:", ridge_gs2.score(X_test2, y_test2))


-----Experiment 2: Year Encoding-----
Ridge Best Params: {'alpha': 13}
Ridge Best CV R^2: 0.8194356349664481
Ridge Train R^2: 0.8458904402462581
Ridge Test R^2: 0.7971467674345594


In [37]:
# TimeSeriesSplit and grid search for Random Forest
rf_model2 = RandomForestRegressor(random_state=30)
rf_gs2 = GridSearchCV(rf_model2, rf_grid, cv=tscv, scoring='r2', n_jobs=-1)
rf_gs2.fit(X_train2, y_train2)

print("RandomForest Best Params:", rf_gs2.best_params_)
print("RandomForest Best CV R^2:", rf_gs2.best_score_)
print("RandomForest Train R^2:", rf_gs2.score(X_train2, y_train2))
print("RandomForest Test R^2:", rf_gs2.score(X_test2, y_test2))

RandomForest Best Params: {'max_depth': 5, 'min_samples_leaf': 4, 'n_estimators': 155}
RandomForest Best CV R^2: 0.829303431930728
RandomForest Train R^2: 0.8486844531578566
RandomForest Test R^2: 0.7831178192600694


In [38]:
# Print Ridge coefficients for inspection
ridge_best2 = ridge_gs2.best_estimator_
coef_df2 = pd.DataFrame({
    'Feature': X_train2.columns,
    'Coefficient': ridge_best2.coef_
})

coef_df2['AbsCoef'] = coef_df2['Coefficient'].abs()
print("Ridge Coefficients (Experiment 2):")
print(coef_df2.sort_values("AbsCoef", ascending=False))

Ridge Coefficients (Experiment 2):
                       Feature  Coefficient   AbsCoef
63                AveragePrice     0.790035  0.790035
7                 type_organic     0.104783  0.104783
25              region_Houston    -0.086112  0.086112
49         region_SanFrancisco     0.079545  0.079545
24  region_HartfordSpringfield     0.077135  0.077135
..                         ...          ...       ...
1                         4046     0.003073  0.003073
42               region_Plains    -0.002671  0.002671
54              region_Spokane    -0.001854  0.001854
0                 Total Volume    -0.000095  0.000095
50              region_Seattle    -0.000042  0.000042

[64 rows x 3 columns]


In Experiment 2 with year encoding, the Ridge regression model achieved a CV $R^2$ of 0.82, with training and test $R^2$ scores respectively of **0.85** and **0.80** and had best parameter: alpha = 13. The Random Forest model achieved a CV $R^2$ of **0.83**, with training and test scores respectively of **0.85** and **0.78** and had best parameters: max_depth=5, min_samples_leaf=4, and n_estimators=155. The Ridge coefficients showed the most influential feature was AveragePrice (0.79), followed by type_organic (0.10) and region_Houston (-0.09).


I compared two approaches for encoding the date in our avocado price forecast. In Experiment 1, I used month encoding by making the month as a categorical variable to capture seasonal patterns in price fluctuations. In Experiment 2, I encoded the year as a categorical variable to reflect broader annual trends. The results showed that experiment 1 slightly perform better compared to the experiment 2 for the month encoding approach. This suggests that month encoding captures seasonal variations more accurately. In conclusion, month encoding appeared more effective for forecasting next week's avocado prices.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Short answer questions

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:6}

The following questions pertain to Lecture 20 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.
3. When studying time series modeling, we explored several ways to encode date information as a feature for the citibike dataset. When we used time of day as a numeric feature, the Ridge model was not able to capture the periodic pattern. Why? How did we tackle this problem? Briefly explain.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 6

A real-world situation where time series data would have unequally spaced time points is the recording of emergency service calls. Emergency service calls occur unpredictably and randomly. Therefore, this leads to irregularly spaced data points rather than fixed intervals.

Creating lagged versions of features struggles with unequally spaced time points. Lagged features assume there is a constant interval between observations. When the intervals vary, these lagged features do not represent consistent durations, thus misleading the model. On the other hand, encoding the date as one or more features provides more flexibility in adapting to irregular spacing. Therefore, lagged features struggle with unequally spaced time points.

When the time of day was encoded as a numeric value, the Ridge model could not capture the periodic behavior inherent in the data. A numeric representation does not reflect the cyclic nature of a day. For example, 23:00 and 01:00 are close in terms of actual time, even though their numeric values appear far apart. To overcome this problem, we transformed the time of day into a categorical variable using one-hot encoding. This encoding allowed the model to learn independent effects for each time category and powerfully capturing the cyclic pattern observed in the citibike dataset.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Computer vision 
rubric={points:6}

The following questions pertain to Lecture 19 on multiclass classification and introduction to computer vision. 

1. How many parameters (coefficients and intercepts) will `sklearn`’s `LogisticRegression()` model learn for a four-class classification problem, assuming that you have 10 features? Briefly explain your answer.
2. In Lecture 19, we briefly discussed how neural networks are sort of like `sklearn`'s pipelines, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?
3. Imagine that you have a small dataset with ~1000 images containing pictures and names of 50 different Computer Science faculty members from UBC. Your goal is to develop a reasonably accurate multi-class classification model for this task. Describe which model/technique you would use and briefly justify your choice in one to three sentences.

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 6

1. For each class the logistic regression learns one coefficient for every feature and one intercept. For a four-class classification problem, assuming that you have 10 features, the model learns a total of 44 parameters because 4 times 10 plus 4 is 44.

2. Neural networks perform several sequential transformations on the data in a manner similar to a pipeline and this allows the early layers to serve as general feature extractors. This property is useful because it enables one to reuse pre-existing representations from a network trained on a large dataset and then adapt the later layers to the specific new task.

3. I would use a pre-trained convolutional neural network like AlexNet and fine tune its final classification layer. This technique is a good choice because the network has already learned robust image features from a large dataset and only the last layers need to be adjusted to the new task.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 2.3 Survival analysis
<hr>

rubric={points:6}

The following questions pertain to Lecture 21 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer? Briefly explain your answer. 
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.3
    
</div>

_Points:_ 6

1. There is a problem with imply labeling customers are "churned" or "not churned" because assigning a binary label fails to capture that customers churn at different times. This standard supervised learning techniques do not consider that many customers are still active at the time of data collection. In doing so, it mistakenly assumes these customers will remain indefinitely. So, this method does not reflect accurately about the true duration of customer engagement.

2. We would expect that customer A who just joined is more likely to leave sooner than customer B. New customers are typically in the period where many users drop out while those who have been with the service for a longer time have already demonstrated their willingness to remain. Therefore, they are at a lower risk during the early phase.

3. If a customer’s survival function is almost flat during a certain period, it indicates that the probability of the customer staying with the service remains nearly constant, suggesting that the risk of churn is very low throughout that time. To put it simply, few additional customers are leaving during this period.

<!-- END QUESTION -->

<br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)